# Modelo de Recurrent Neural Network

Importaciones necesarias

In [55]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchsummary import summary
from torchviz import make_dot

In [56]:

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(64 * 56 * 56, 128)
        self.fc2 = nn.Linear(128, 2)

    def forward(self, x):
        x = self.pool(nn.functional.relu(self.conv1(x)))
        x = self.pool(nn.functional.relu(self.conv2(x))) 
        x = x.view(-1, 64 * 56 * 56)
        x = nn.functional.relu(self.fc1(x))
        x = self.fc2(x)
        return x

Se crea el Modelo de CNN

In [57]:
model = CNN()

Se carga el dataset de training y de test para entrenar el modelo

In [58]:
# se cargan todas las imagenes en la carpeta PetImages y luego se dividen en train y test

# transform = transforms.Compose([transforms.Resize((28, 28)), transforms.ToTensor()])
# dataset = torchvision.datasets.ImageFolder(root='./PetImages', transform=transform)

# train_dataset, test_dataset = torch.utils.data.random_split(dataset, [int(len(dataset) * 0.8), len(dataset) - int(len(dataset) * 0.8)])
# train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True)
# test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=128, shuffle=True)

# import os
# import tensorflow as tf
# import numpy as np
# import cv2 as ocv
# from sklearn import random
# from sklearn.model_selection import train_test_split

# def load_data(dir, class_names, num_of_img_per_class, input_shape):
#     classes = class_names
#     num_of_img_per_class = num_of_img_per_class
    
#     data = []
#     labels = []
    
#     class_counter = 1
#     for category in os.listdir(dir):
        
#         if class_counter==1:
#             print("Loading Data:", end=" ")
#         new_dir = os.path.join(dir,category)
        
#         for img in os.listdir(new_dir):
#             if len(data)<(num_of_img_per_class * class_counter):
#                 img_path = os.path.join(new_dir,img)
#                 if (np.all(np.array(ocv.imread(img_path,0)))==0):
#                     data.append(ocv.resize(ocv.imread(img_path,1),(input_shape[0],input_shape[1])))
#                     labels.append(classes.index(category))
#                     if len(data)/(num_of_img_per_class*2)*100%20==0:
#                         print(int(len(data)/(num_of_img_per_class*2)*100),"%", end="  ")
                        
#         class_counter=class_counter+1
        
#         if class_counter==3:
#             print(". Complete.", end=" ")
            
#     combined = list(zip(data,labels))
#     random.shuffle(combined)
#     data, labels = zip(*combined)
#     Data = np.array(data)/255
#     Labels = np.array(labels)
#     n_col = Data.shape[1]
#     n_rows = Labels.shape[0]
#     return train_test_split(Data, Labels)


# dir = './PetImages'
# classes = ['Cat','Dog']
# num_of_img_per_class =1000
# input_shape = (224, 224, 3)
# x_train, x_test, y_train, y_test = load_data(dir, classes, num_of_img_per_class, input_shape)

# model = DenseNetModel(input_shape)

# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# model.fit(x_train, y_train, epochs = 20, batch_size = 32)
# test_set_accuracy = model.evaluate(x_test, y_test)[1] * 100
# print('model accuracy =', test_set_accuracy, '%')




In [59]:
import os
import torch
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split
from PIL import Image

# Transformaciones que se aplicarán a las imágenes
transform = transforms.Compose([transforms.Resize((224, 224)),
                                transforms.Grayscale(),
                                transforms.ToTensor()])

# Función para verificar si una imagen es válida
def is_valid_image(filename):
    try:
        with Image.open(filename) as img:
            img.verify()
            if os.path.getsize(filename) > 0:
                return True
            else:
                return False
    except:
        return False

# Iterar a través de todas las imágenes en el conjunto de datos
for root, dirs, files in os.walk("./PetImages"):
    for file in files:
        filename = os.path.join(root, file)
        
        # Verificar si la imagen es válida
        if not is_valid_image(filename):
            print(f"La imagen {filename} es inválida. Eliminando...")
            os.remove(filename)

# Crear un objeto ImageFolder para cargar los datos
dataset = ImageFolder(root='./PetImages', transform=transform, is_valid_file=is_valid_image)
print(len(dataset))

# # Eliminar las imágenes corruptas
# indices = [i for i, (image, label) in enumerate(dataset) if not is_valid_image(dataset.imgs[i][0])]
# print(ind)
#  is_valid_image(dataset.imgs[i][0])]
# dataset = torchprint(len(dataset))

# Dividir el dataset en entrenamiento y prueba
train_size = int(len(dataset) * 0.8)
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

print(train_size)
print(test_size)

# Crear los dataloaders
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=True)


24996
19996
5000


Se entrena el modelo para que pueda hacer las predicciones

In [60]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
num_epochs = 10
for epoch in range(num_epochs):
    
    for i, (images, labels) in enumerate(train_loader):
        try:
            # Clean gradient before new batch
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            # Call backward propagation
            loss.backward()
            optimizer.step()
            if (i+1) % 100 == 0:
                print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, i+1, len(train_loader), loss.item()))
        except Exception as e:
            continue    

Epoch [1/10], Step [100/157], Loss: 0.6943
Epoch [2/10], Step [100/157], Loss: 0.6938
Epoch [3/10], Step [100/157], Loss: 0.6925
Epoch [4/10], Step [100/157], Loss: 0.6597
Epoch [5/10], Step [100/157], Loss: 0.5339
Epoch [6/10], Step [100/157], Loss: 0.4397
Epoch [7/10], Step [100/157], Loss: 0.4458
Epoch [8/10], Step [100/157], Loss: 0.3921
Epoch [9/10], Step [100/157], Loss: 0.2709
Epoch [10/10], Step [100/157], Loss: 0.1220


Se evalua el modelo con el dataset de test para ver su precision

In [62]:
# Test the model
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum()
    print('Accuracy of the network on the test images: {} %'.format(100 * correct / total))


Accuracy of the network on the test images: 69.73999786376953 %


Debido a que el entrenamiento del modelo es bastante tardado, colocamos el accuracy obtenido:
    Accuracy of the network on the test images: 69.73999786376953 %